In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
#pip install netaddr

In [4]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [5]:
wed_data = pd.read_pickle(r'C:\\Users\\jghadermazi\\Box\\GAN Project\\Sample data\\Friday_Packet_Level_Data.pkl')

In [6]:
df1 = wed_data.copy()

In [7]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [8]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499432653.990571000,192.168.10.9,205.174.165.73,1841,8080,13099,0,00c1b114eb31b8ac6f1d1f6c0800,450000346c8f40008006508bc0a80a09cdaea549,07311f905c691851000000008002200075eb0000020405...,00,fwd,1.0,12.0
1,1499432653.991397000,205.174.165.73,192.168.10.9,8080,1841,13099,0,b8ac6f1d1f6c00c1b114eb310800,45000034000040003e06ff1acdaea549c0a80a09,1f90073113dd44cb5c69185280127210cb220000020405...,00,bwd,1.0,12.0
2,1499432653.991520000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000286c90400080065096c0a80a09cdaea549,07311f905c69185213dd44cc501001007d050000,000000000000,fwd,1.0,12.0
3,1499432653.992542000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000ea6c91400080064fd3c0a80a09cdaea549,07311f905c69185213dd44cc50180100307c0000,474554202f6170692f706f703f626f7469643d6d697461...,fwd,1.0,12.0
4,1499432653.993057000,205.174.165.73,192.168.10.9,8080,1841,13099,1,b8ac6f1d1f6c00c1b114eb310800,45000028822a40003e067cfccdaea549c0a80a09,1f90073113dd44cc5c691914501000ed7c560000,000000000000,bwd,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260217,1499454300.212460000,172.217.12.205,192.168.10.14,443,58093,280715,3982,b8ac6f3607ee00c1b114eb310800,450000f7dfe60000340621beacd90ccdc0a80a0e,01bbe2eda84cbf47d787ec2350180169decb0000,17030300ca0000000000000003b9423a0b5901a1d34005...,bwd,0.0,0.0
5260218,1499454300.212688000,172.217.12.205,192.168.10.14,443,58093,280715,4189,b8ac6f3607ee00c1b114eb310800,4500019ddfe7000034062117acd90ccdc0a80a0e,01bbe2eda84cc016d787ec2350180169e0870000,1703030170000000000000000427cfa8c91d47cda4d459...,bwd,0.0,0.0
5260219,1499454300.212736000,172.217.12.205,192.168.10.14,443,58093,280715,4562,b8ac6f3607ee00c1b114eb310800,450000acdfe8000034062207acd90ccdc0a80a0e,01bbe2eda84cc18bd787ec235018016903ef0000,170303007f000000000000000529465b5b09e2c6a7bf19...,bwd,0.0,0.0
5260220,1499454300.212738000,172.217.12.205,192.168.10.14,443,58093,280715,4694,b8ac6f3607ee00c1b114eb310800,45000056dfe900003406225cacd90ccdc0a80a0e,01bbe2eda84cc20fd787ec2350180169e55a0000,170303002900000000000000066521f26029ac678a3244...,bwd,0.0,0.0


In [9]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [10]:
df1['Class Label'].value_counts()

0.0     3661387
14.0    1265657
13.0     323307
12.0       9871
Name: Class Label, dtype: int64

In [11]:
df = df1[df1['Class Label']== 12]

In [12]:
df = df.reset_index(inplace = False)

In [13]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,0,1499432653.990571000,192.168.10.9,205.174.165.73,1841,8080,13099,0,00c1b114eb31b8ac6f1d1f6c0800,450000346c8f40008006508bc0a80a09cdaea549,07311f905c691851000000008002200075eb0000020405...,00,fwd,1.0,12.0
1,1,1499432653.991397000,205.174.165.73,192.168.10.9,8080,1841,13099,0,b8ac6f1d1f6c00c1b114eb310800,45000034000040003e06ff1acdaea549c0a80a09,1f90073113dd44cb5c69185280127210cb220000020405...,00,bwd,1.0,12.0
2,2,1499432653.991520000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000286c90400080065096c0a80a09cdaea549,07311f905c69185213dd44cc501001007d050000,000000000000,fwd,1.0,12.0
3,3,1499432653.992542000,192.168.10.9,205.174.165.73,1841,8080,13099,1,00c1b114eb31b8ac6f1d1f6c0800,450000ea6c91400080064fd3c0a80a09cdaea549,07311f905c69185213dd44cc50180100307c0000,474554202f6170692f706f703f626f7469643d6d697461...,fwd,1.0,12.0
4,4,1499432653.993057000,205.174.165.73,192.168.10.9,8080,1841,13099,1,b8ac6f1d1f6c00c1b114eb310800,45000028822a40003e067cfccdaea549c0a80a09,1f90073113dd44cc5c691914501000ed7c560000,000000000000,bwd,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,9866,1499436122.810043000,205.174.165.73,192.168.10.8,8080,3051,27753,1,b8ac6f3608f500c1b114eb310800,45000028f42840003e060affcdaea549c0a80a08,1f900bebb148f65c10191d00501000ed71050000,000000000000,bwd,1.0,12.0
9867,9867,1499436122.902573000,205.174.165.73,192.168.10.8,8080,3051,27753,1,b8ac6f3608f500c1b114eb310800,450000a8f42940003e060a7ecdaea549c0a80a08,1f900bebb148f65c10191d00501800eda5730000,485454502f312e3120323030204f4b0d0a446174653a20...,bwd,1.0,12.0
9868,9868,1499436122.903154000,192.168.10.8,205.174.165.73,3051,8080,27753,199,00c1b114eb31b8ac6f3608f50800,450000282ea0400080068e87c0a80a08cdaea549,0beb1f9010191d00b148f6dc5011400931680000,000000000000,fwd,1.0,12.0
9869,9869,1499436122.903712000,205.174.165.73,192.168.10.8,8080,3051,27753,129,b8ac6f3608f500c1b114eb310800,45000028f42a40003e060afdcdaea549c0a80a08,1f900bebb148f6dc10191d01501100ed70830000,000000000000,bwd,1.0,12.0


In [14]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [15]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [16]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [17]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [18]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,192.168.10.9,205.174.165.73,1841,8080,1499432653.990571000,13099,0,fwd,"[0.0, 0.20392156862745098, 0.4235294117647059,..."
1,205.174.165.73,192.168.10.9,8080,1841,1499432653.991397000,13099,0,bwd,"[0.0, 0.20392156862745098, 0.0, 0.0, 0.2509803..."
2,192.168.10.9,205.174.165.73,1841,8080,1499432653.991520000,13099,1,fwd,"[0.0, 0.1568627450980392, 0.4235294117647059, ..."
3,192.168.10.9,205.174.165.73,1841,8080,1499432653.992542000,13099,1,fwd,"[0.0, 0.9176470588235294, 0.4235294117647059, ..."
4,205.174.165.73,192.168.10.9,8080,1841,1499432653.993057000,13099,1,bwd,"[0.0, 0.1568627450980392, 0.5098039215686274, ..."
...,...,...,...,...,...,...,...,...,...
9866,205.174.165.73,192.168.10.8,8080,3051,1499436122.810043000,27753,1,bwd,"[0.0, 0.1568627450980392, 0.9568627450980393, ..."
9867,205.174.165.73,192.168.10.8,8080,3051,1499436122.902573000,27753,1,bwd,"[0.0, 0.6588235294117647, 0.9568627450980393, ..."
9868,192.168.10.8,205.174.165.73,3051,8080,1499436122.903154000,27753,199,fwd,"[0.0, 0.1568627450980392, 0.1803921568627451, ..."
9869,205.174.165.73,192.168.10.8,8080,3051,1499436122.903712000,27753,129,bwd,"[0.0, 0.1568627450980392, 0.9568627450980393, ..."


In [19]:
streams_lst = list(df['Stream Index'].unique())

In [20]:
len(streams_lst)

736

In [21]:
# 70% of benign data for train, 30% for test benign

In [22]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [23]:
#len(imlist)

In [24]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [25]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [26]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [27]:
#len(streams_lst1)

In [28]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,192.168.10.9,205.174.165.73,1841,8080,1499432653.990571000,13099,0,fwd,"[0.0, 0.20392156862745098, 0.4235294117647059,..."
1,205.174.165.73,192.168.10.9,8080,1841,1499432653.991397000,13099,0,bwd,"[0.0, 0.20392156862745098, 0.0, 0.0, 0.2509803..."
2,192.168.10.9,205.174.165.73,1841,8080,1499432653.991520000,13099,1,fwd,"[0.0, 0.1568627450980392, 0.4235294117647059, ..."
3,192.168.10.9,205.174.165.73,1841,8080,1499432653.992542000,13099,1,fwd,"[0.0, 0.9176470588235294, 0.4235294117647059, ..."
4,205.174.165.73,192.168.10.9,8080,1841,1499432653.993057000,13099,1,bwd,"[0.0, 0.1568627450980392, 0.5098039215686274, ..."
...,...,...,...,...,...,...,...,...,...
9866,205.174.165.73,192.168.10.8,8080,3051,1499436122.810043000,27753,1,bwd,"[0.0, 0.1568627450980392, 0.9568627450980393, ..."
9867,205.174.165.73,192.168.10.8,8080,3051,1499436122.902573000,27753,1,bwd,"[0.0, 0.6588235294117647, 0.9568627450980393, ..."
9868,192.168.10.8,205.174.165.73,3051,8080,1499436122.903154000,27753,199,fwd,"[0.0, 0.1568627450980392, 0.1803921568627451, ..."
9869,205.174.165.73,192.168.10.8,8080,3051,1499436122.903712000,27753,129,bwd,"[0.0, 0.1568627450980392, 0.9568627450980393, ..."


In [29]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Friday_botnet_all/'

for PKT in range(2, 16):
    
    file_name = f'friday_botnet_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df1 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df1.to_csv(file_path, index=False)

Total number of flows in class: 736
